# OLX portal web page scraper

## Mount drive

In [1]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Import libraries

In [2]:
# from bs4 import BeautifulSoup

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import pandas as pd
import numpy  as np
import re
import datetime
import os,glob
import json
from json.decoder import JSONDecodeError
from tqdm import tqdm
import re


## Beautify

In [3]:
start_url= 'https://www.olx.pl/nieruchomosci/mieszkania/wynajem/gdansk/q-Gdansk-centrum/?search%5Bfilter_enum_furniture%5D%5B0%5D=yes&search%5Bfilter_enum_rooms%5D%5B0%5D=two&view=list' 

In [5]:
# makes the source tree format like

def beautify(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)

    return BeautifulSoup(page, "html.parser")

In [6]:
flats = beautify(start_url)

## Scrap table and put to lists

In [7]:
table = flats.find('table', {'class': "fixed offers breakword redesigned"})
for row in table.findAll("href"):
  print(row.get)

In [8]:
# giving all th erow also empty no text

table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [d.text.strip() for d in td]
    res.append(row)

lets remove duplicates

In [11]:
result = [] 
for nr, link in enumerate(table.find_all('a')):
  if link.get('href').__contains__('/oferta/'): # specific text which needs to be in the link
    if link.get('href') not in result:  # avoiding duplicates
      result.append(link.get('href'))
result

['https://www.olx.pl/d/oferta/piekne-2-u-pok-mieszkanie-blisko-lasu-i-centrum-miejsce-postojowe-CID3-IDL4pWX.html#fa57d769ac;promoted',
 'https://www.olx.pl/d/oferta/apartament-4-os-centrum-wrzeszcz-na-doby-krotki-termin-CID3-IDKLQ6o.html#fa57d769ac;promoted',
 'https://www.olx.pl/d/oferta/kameralne-mieszkanie-pracownicze-blisko-centrum-CID3-IDAyOAq.html#fa57d769ac',
 'https://www.olx.pl/d/oferta/dwupokojowe-mieszkanie-w-samym-centrum-gdansk-wrzeszcz-ul-jesionowa-CID3-IDHjxyH.html#fa57d769ac',
 'https://www.olx.pl/d/oferta/2-pokojowe-mieszkanie-w-centrum-gdanska-nowy-apartamentowiec-CID3-IDL5Zei.html#fa57d769ac',
 'https://www.olx.pl/d/oferta/mieszkanie-stare-miasto-starowka-srodmiescie-centrum-CID3-IDIlJfu.html#fa57d769ac;promoted',
 'https://www.otodom.pl/pl/oferta/dwupokojowy-apartament-w-centrum-gdanska-wrzeszcza-ID4cYQr.html?',
 'https://www.olx.pl/d/oferta/neptun-park-centrum-apartament-gdansk-CID3-IDKZUlL.html#fa57d769ac',
 'https://www.olx.pl/d/oferta/sloneczne-2-pokoje-gdansk-

In [12]:
# and funcion doing removing extra spaces and new lines

def remove_extra(temp):
  temp = re.sub(r'\n+', ' ', temp) #removing extra new lines
  temp = re.sub(r' +', ' ', temp)  #removing extra spaces
  return temp

In [13]:
# retrieving info with regex to the tables
l_cena=[]
l_date_=[]
l_descr=[]

for nr, i in enumerate(res):
  if len(i)>3:
    cena= re.findall(r".+zł", i[0])
    date_= re.findall(r"\d+\s{2}\w{3}|.*\d+:\d+", i[0])
    descr= i=remove_extra(i[0])

    l_cena.append(cena)
    l_date_.append(date_)
    l_descr.append(descr)

    # print( cena[0].ljust(15), date_[0].ljust(15), descr)




# Print results from lists

In [14]:
for nr, i in enumerate(result):
  print(str(nr).ljust(2),
        result[nr].ljust(130),
        l_cena[nr],
        l_date_[nr],
        l_descr[nr])

0  https://www.olx.pl/d/oferta/piekne-2-u-pok-mieszkanie-blisko-lasu-i-centrum-miejsce-postojowe-CID3-IDL4pWX.html#fa57d769ac;promoted ['2 000 zł'] ['dzisiaj 12:34'] Piękne, 2-u pok. mieszkanie blisko lasu i Centrum + miejsce postojowe Mieszkania » Wynajem 2 000 zł Gdańsk, Piecki-Migowo dzisiaj 12:34
1  https://www.olx.pl/d/oferta/apartament-4-os-centrum-wrzeszcz-na-doby-krotki-termin-CID3-IDKLQ6o.html#fa57d769ac;promoted           ['300 zł'] ['wczoraj 18:44'] Apartament 4 os centrum WRZESZCZ- na doby/krótki termin Mieszkania » Wynajem 300 zł Gdańsk, Wrzeszcz wczoraj 18:44
2  https://www.olx.pl/d/oferta/kameralne-mieszkanie-pracownicze-blisko-centrum-CID3-IDAyOAq.html#fa57d769ac                           ['700 zł'] ['wczoraj 17:08'] Kameralne mieszkanie pracownicze - blisko centrum Mieszkania » Wynajem 700 zł Gdańsk, Śródmieście wczoraj 17:08
3  https://www.olx.pl/d/oferta/dwupokojowe-mieszkanie-w-samym-centrum-gdansk-wrzeszcz-ul-jesionowa-CID3-IDHjxyH.html#fa57d769ac       ['2 000 zł'

Update to tables library-pandas need that to avoid write errors

In [15]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3 MB 4.7 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


# Saving results to file

In [16]:
plots_olx_pd = pd.DataFrame({'web_link' : result,
                               'price' : l_cena,
                               'time' : l_date_,
                               'Description': l_descr})

In [ ]:
# plots_olx_pd

In [23]:
# generating file name with timestamp info for later comparison
def gen_h5_file_with_timestamp(name='name'):
    timestamp = datetime.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".h5"
    return file_name


In [24]:
file_name= gen_h5_file_with_timestamp(name='mieszkanie_gdansk_centrum_olx_')
file_name

'mieszkanie_gdansk_centrum_olx_Tuesday24August202108-11-22-14.h5'

In [25]:
plots_olx_pd.to_hdf('/content/drive/MyDrive/olx-mieszkanie/saved/'+file_name, key='df', mode='w') 

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['web_link', 'price', 'time', 'Description'], dtype='object')]

  encoding=encoding,


# Checking changes since last time

In [20]:
# preparing list of existing(previously created by scrapping) pandas linkedin .h5 files
search_dir = "/content/drive/MyDrive/olx-mieszkanie/saved/" #directory where from where h5 is read s
def list_per_time(search_dir=search_dir):
    files = [x for x in (filter(os.path.isfile, glob.glob(search_dir + "mieszk*.h5"))) ] 
    files_sorted= sorted(files, key=os.path.getmtime)
    print(files_sorted[:10])
    return files_sorted

In [21]:
# loading last( creation time) two dfs for comparison
def read_two_last_dfs():
    files_sorted=list_per_time()
    f1=files_sorted[-2:-1]
    f2=files_sorted[-1:]
    df1=pd.read_hdf(f1[0], 'df')
    df2=pd.read_hdf(f2[0], 'df')
    return df1, df2

In [26]:
df1, df2=read_two_last_dfs()

['/content/drive/MyDrive/olx-mieszkanie/saved/mieszkanie_gdansk_centrum_olx_Tuesday24August202108-11-21-09.h5', '/content/drive/MyDrive/olx-mieszkanie/saved/mieszkanie_gdansk_centrum_olx_Tuesday24August202108-11-22-14.h5']


In [27]:
print('=============links to new plots since last checking=============\n')
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df1['web_link'])]):
    print(x, '-->',item )

=============links to new plots since last checking=============



Checking differece to 18/08/2021

In [28]:
new_plots=[]
df_ref = pd.read_hdf('/content/drive/MyDrive/olx-scrap/scraped/agro_plots_olx_Wednesday18August202108-10-13-54.h5', 'df')
print('=============links to new plots since 18/08/2021 =============\n')
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df_ref['web_link'])]):
  new_plots.append(item)
  print(x, '-->',item )

=============links to new plots since 18/08/2021 =============

0 --> https://www.olx.pl/d/oferta/piekne-2-u-pok-mieszkanie-blisko-lasu-i-centrum-miejsce-postojowe-CID3-IDL4pWX.html#fa57d769ac;promoted
1 --> https://www.olx.pl/d/oferta/apartament-4-os-centrum-wrzeszcz-na-doby-krotki-termin-CID3-IDKLQ6o.html#fa57d769ac;promoted
2 --> https://www.olx.pl/d/oferta/kameralne-mieszkanie-pracownicze-blisko-centrum-CID3-IDAyOAq.html#fa57d769ac
3 --> https://www.olx.pl/d/oferta/dwupokojowe-mieszkanie-w-samym-centrum-gdansk-wrzeszcz-ul-jesionowa-CID3-IDHjxyH.html#fa57d769ac
4 --> https://www.olx.pl/d/oferta/2-pokojowe-mieszkanie-w-centrum-gdanska-nowy-apartamentowiec-CID3-IDL5Zei.html#fa57d769ac
5 --> https://www.olx.pl/d/oferta/mieszkanie-stare-miasto-starowka-srodmiescie-centrum-CID3-IDIlJfu.html#fa57d769ac;promoted
6 --> https://www.otodom.pl/pl/oferta/dwupokojowy-apartament-w-centrum-gdanska-wrzeszcza-ID4cYQr.html?
7 --> https://www.olx.pl/d/oferta/neptun-park-centrum-apartament-gdansk-CID3-

What will be diffrence if df1 and 2 were changed places (what dissapeared...)

In [30]:
dissapeared=[]
df_ref = pd.read_hdf('//content/drive/MyDrive/olx-mieszkanie/saved/mieszkanie_gdansk_centrum_olx_Tuesday24August202108-11-21-09.h5', 'df')
print('=============those dissaperarewd since 18/06/2021 =============\n')
for x,item in enumerate(df_ref.web_link.values[~df_ref['web_link'].isin(df2['web_link'])]):
  dissapeared.append(item)
  print(x, '-->',item,  )
  print(df_ref[df_ref.web_link==item].Description.values)

=============those dissaperarewd since 18/06/2021 =============



Checking if new plots are only the change  from promoted?

In [31]:
for plot in dissapeared:
  for new_p in new_plots:
    if plot.__contains__(new_p):
      print('is-->',new_p ,'  --was--->',plot)

In [32]:
def cosmetic_ad_change_check():
  
  for plot in dissapeared:
    for new_p in new_plots:
      if plot.__contains__(new_p):
        print('is-->',new_p ,'  --was--->',plot)

# Report

In [33]:
#@title Overall report { display-mode: "form" }

df_ref_link = "/content/drive/MyDrive/olx-mieszkanie/saved/mieszkanie_gdansk_centrum_olx_Tuesday24August202108-11-21-09.h5" #@param ["'/content/drive/MyDrive/olx-scrap/scraped/agro_plots_olx_Wednesday18August202108-10-13-54.h5'"] {allow-input: true}


df_ref = pd.read_hdf(df_ref_link, 'df')

new_plots=[]
dissapeared=[]
ti= df_ref_link[-26: -12]
print('=============links to new plots since {} ============='.format(ti))
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df_ref['web_link'])]):
  new_plots.append(item)
  print(x, '-->',item )

print('\n','=============those dissapeared since {} ============='.format(ti))
for x,item in enumerate(df_ref.web_link.values[~df_ref['web_link'].isin(df2['web_link'])]):
  dissapeared.append(item)
  print(x, '-->',item)
  print(df_ref[df_ref.web_link==item].Description.values)

print('\n','Remarks:')
cosmetic_ad_change_check()




=============links to new plots since 24August202108 =============

 =============those dissapeared since 24August202108 =============

 Remarks:
